# Task 1: Credit Card Fraud Detection with Neural Networks

This notebook implements both baseline and ANN models for fraud detection.

## 1. Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.linear_model import LogisticRegression

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

ModuleNotFoundError: No module named 'tensorflow.python'

## 2. Load and Explore Data

In [ ]:
# 1. Load data
df = pd.read_csv("../data/creditcard.csv")
print(df.shape)
print(df['Class'].value_counts())   # 0 = normal, 1 = fraud

## 3. Prepare Data

In [ ]:
# 2. Features & labels
X = df.drop('Class', axis=1).values
y = df['Class'].values

# 3. Train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. Scale features (important for ANNs)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## 4. Baseline Model - Logistic Regression

In [ ]:
baseline = LogisticRegression(max_iter=1000, class_weight="balanced")
baseline.fit(X_train, y_train)

y_pred_base = baseline.predict(X_test)
y_proba_base = baseline.predict_proba(X_test)[:, 1]

print("BASELINE Logistic Regression")
print(classification_report(y_test, y_pred_base, digits=4))
print("AUC:", roc_auc_score(y_test, y_proba_base))

cm = confusion_matrix(y_test, y_pred_base)
print("Confusion matrix:\n", cm)

## 5. Simple ANN Model

In [ ]:
input_dim = X_train.shape[1]

model = keras.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(32, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")  # binary classification
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=2048,
    verbose=1
)

## 6. Training Visualization

In [ ]:
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.legend()
plt.title("ANN – Loss")
plt.show()

plt.plot(history.history["accuracy"], label="train_acc")
plt.plot(history.history["val_accuracy"], label="val_acc")
plt.legend()
plt.title("ANN – Accuracy")
plt.show()

## 7. ANN Model Evaluation

In [ ]:
y_proba_ann = model.predict(X_test).ravel()
y_pred_ann = (y_proba_ann >= 0.5).astype(int)

print("ANN model")
print(classification_report(y_test, y_pred_ann, digits=4))
print("AUC:", roc_auc_score(y_test, y_proba_ann))

cm_ann = confusion_matrix(y_test, y_pred_ann)
print("Confusion matrix:\n", cm_ann)

## 8. Experimental Variations (Model B: Deeper Network)

In [ ]:
# Model B: 64-32-16 (deeper network)
model_b = keras.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(64, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model_b.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history_b = model_b.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=2048,
    verbose=1
)

# Evaluate Model B
y_proba_b = model_b.predict(X_test).ravel()
y_pred_b = (y_proba_b >= 0.5).astype(int)

print("\nModel B (64-32-16)")
print(classification_report(y_test, y_pred_b, digits=4))
print("AUC:", roc_auc_score(y_test, y_proba_b))

## 9. Experimental Variations (Model C: With Dropout)

In [ ]:
# Model C: Add Dropout for regularization
model_c = keras.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(32, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model_c.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history_c = model_c.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=2048,
    verbose=1
)

# Evaluate Model C
y_proba_c = model_c.predict(X_test).ravel()
y_pred_c = (y_proba_c >= 0.5).astype(int)

print("\nModel C (32-16 with Dropout)")
print(classification_report(y_test, y_pred_c, digits=4))
print("AUC:", roc_auc_score(y_test, y_proba_c))

## 10. Final Model Comparison

In [ ]:
print("=== FINAL MODEL COMPARISON ===\n")
print(f"Baseline (Logistic):     AUC = {roc_auc_score(y_test, y_proba_base):.4f}")
print(f"Model A (32-16):         AUC = {roc_auc_score(y_test, y_proba_ann):.4f}")
print(f"Model B (64-32-16):      AUC = {roc_auc_score(y_test, y_proba_b):.4f}")
print(f"Model C (32-16+Dropout): AUC = {roc_auc_score(y_test, y_proba_c):.4f}")